In [ ]:
# import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
bathy = np.loadtxt("Varna_topo_channels.txt")  # Bathymetry file path
coper_data_path = r"C:\Users\M.Novikov\Nayka\BROM\data\Varna"  # Data directory path

### Domain

In [ ]:
da_bathy = xr.DataArray(bathy)

In [ ]:
# Check z value
da_bathy.min()

In [ ]:
plt.pcolormesh(da_bathy)
plt.colorbar()

In [ ]:
domain = {'lon':[27.640,27.640,27.947,27.947], 'lat':[43.177,43.214,43.214,43.177]}

In [ ]:
# Just for fun - input data and gomain location

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator(central_longitude=180))
# Coper data
ax.pcolormesh(coper_data["longitude"], coper_data["latitude"], coper_data["uo"][0,0,...], transform=ccrs.PlateCarree())
# East domain
ax.plot(domain['lon'], domain['lat'], marker='o', transform=ccrs.PlateCarree())
ax.fill(domain['lon'], domain['lat'], color='c', transform=ccrs.PlateCarree(), alpha=0.4)
ax.set_extent([27.8, 28, 43.1, 43.3])
ax.add_feature(cfeature.LAND, zorder=1, alpha = 0.2)
ax.add_feature(cfeature.RIVERS, zorder=1)
ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=1)
ax.coastlines(zorder=1)
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, zorder=1)

# plt.savefig('Images/ЕПЦС_NEMO_катра.png', dpi=600, bbox_inches='tight')

### Coper data

In [ ]:
coper_data = xr.open_mfdataset(f"{coper_data_path}\*24*")  # Load all downloaded CMEMS files (cur, tem, sal and ssh)

In [ ]:
coper_data

### Boundary creation

In [ ]:
# Boundary mesh
x_coord = 27.947
y_coord = np.linspace(43.177, 43.214, 42)
z_coord = -1*np.arange(0, da_bathy.min()-0.5, -0.5)

In [ ]:
# data extraction
bdy_data = coper_data.interp(latitude=y_coord, longitude=x_coord, method="nearest" ,kwargs={"fill_value": "extrapolate"},)

In [ ]:
bdy_data["uo"][2,...].plot()
plt.ylim(0,25)

In [ ]:
# Raw interpolation
bdy_data = bdy_data.interp(depth=z_coord)
bdy_data["uo"][2].plot()

In [ ]:
# NaN fillings ind smoothing
bdy_data = bdy_data.interpolate_na(dim="latitude", method="nearest", fill_value="extrapolate")
bdy_data = bdy_data.interpolate_na(dim="depth", method="nearest", fill_value="extrapolate")
bdy_data = bdy_data.rolling(latitude=7).mean()
bdy_data = bdy_data.interpolate_na(dim="latitude", method="nearest", fill_value="extrapolate")
bdy_data["depth"] = -1*bdy_data["depth"]

In [ ]:
# At last
bdy_data["zos"][-1].plot()

In [ ]:
# bdy_data.to_netcdf(f"{coper_data_path}\Varna_bdy_05-24.nc")

In [ ]:
# Test the file
bdy_data = xr.open_dataset(f"{coper_data_path}\Varna_bdy_05-24.nc")
bdy_data